# Py #3 - Print vs Logging

Merhabalar, Python dilinin kullanışlı özelliklerini incelediğimiz serinin üçüncü yazısında, ürün seviyesindeki yazılım projelerinin neredeyse tamamında kullanılan **logging** kütüphanesini, kullanımlarını ve **print** fonksiyonundan farklarını/avantajlarını inceleyeceğiz.

Hangi yazılım dilini öğreniyor olursak olalım, neredeyse hepimizin öğrendiği ve yazdığı ilk şey aynıdır:

In [1]:
print("Hello World!")

Hello World!


## Neden Logging?

Ancak meslek hayatımızda bir ürün geliştirirken, **print** fonksiyonu pek de kullanışlı değildir. 

- Kodunuzun doğru çalışıyor olduğunu periyodik olarak görmek isteyebilirsiniz.
- Hata mesajlarınızın hangi dosyanın hangi satırında, hangi fonksiyon içerisinde ve saat kaçta oluştuğunu bilmek isteyebilirsiniz. 
- Tutmak istediğiniz kayıtları önem derecesine göre ayırt edebilirsiniz.

Bunlar **logging** kütüphanesinin size sağladığı kolaylıklardır. Ancak bunların dışında 2 kritik önemi daha vardır:

- Kodunuzun diğer python araçları veya scriptleri tarafından içe aktarılması söz konusuysa, print kullarak bir şeyler stdout'a yazdırması pratik bir fikir değildir çünkü kullanan kişi bu mesajların nereden geldiğini bilemeyecektir.
- Ayrıca, yüksek ihtimalle kodunuzu Dockerize ederek(Yani bir Docker imajına çevirerek ve bir Docker container'ı içerisinde) çalıştırıyor olacaksınız. Bunun anlamı, print kullanarak yazdırdığınız hiçbir satırı container loglarında göremeyecek olmanızdır, çünkü bu esnada konsola erişiminiz yoktur. Print fonksiyonu, yalnızca konsola yazdırma işlemini yapar.

## Logging ile Kayıt Dosyası Oluşturma

Logging kütüphanesinin örnek bir kullanımı aşağıdaki gibidir.

In [2]:
# logging_file_example.py isimli bir Python dosyası oluşturalım.

import logging
 
logging.basicConfig(filename="logfile.txt", 
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    filemode='w',
                    level = logging.ERROR)
 
logger = logging.getLogger()
 
try:
    logger.debug("Process is started")
    result = 5 / 0
except:
    logger.error("Exception thrown", exc_info = True)
    print("You will not see this line in log file, just a simple output")

You will not see this line in log file, just a simple output


Bu dosyayı oluşturup "python logging_file_example.py" komutuyla çalıştırdığımızda sadece bir satırlık bir çıktı gördük. Bu, **print** fonksiyonunun konsola yazdırdığı ekran çıktısıdır. **logging** kütüphanesinin çıktısı ise dosyayı çalıştırdığınız dizindedir. **basicConfig** parametrelerinde belirttiğiniz **logfile.txt** isimli dosya oluşturulmuş ve bu çıktı oraya kaydedilmiştir. Bu çıktıyı inceleyelim:

![title](file_output.png)

Ekrana yazmasını istediğimiz cümle, ve kayıt tipi, hatanın tam olarak hangi dizinde/dosyada olduğu, kaçıncı satırda olduğu, hataya sebep olan kod kısmı, hatanın türü ve mesajı tam olarak yazılmıştır.

Aynı detayları **print** fonksiyonu ile yazdırabilmek için kullandığınız her nokta bu tanımları tek tek kendiniz yapmalısınız. Ancak **logging** kütüphanesi için **basicConfig** içerisinde bir kere yapmamız yeterli.

Yazdığımız kodu biraz daha derinlemesine inceleyelim. Örneğin 14. satırdaki **debug** kaydı txt dosyamızda yok. Peki neden? Bunun cevabı **basicConfig** içerisindeki **level** parametresidir. Bu parametreyi **error** olarak belirttiğimiz için, önem düzeyi olarak daha aşağıda kalan hiçbir logu dosyaya yazmayacaktır.

## Log seviyeleri nelerdir?

Önem düzeyi artan şekilde log seviyeleri şunlardır:

**Debug**: Program çalışmaya devam ederken ortaya çıkan beklenmedik bir çıktıyı incelemek/düzeltmek için kullanırız.

**Info**: Yazılımımızın sağlıklı bir biçimde akışını gözlemlemek için kullanırız.

**Warning**: Beklenmedik hata vb. bir durumu gözlemlemek istediğimizde kullanırız.

**Error**: Programda bir problem oluştuğu zaman, genellikle program bazı komutları çalıştıramadığı zaman kullanırız.

**Critical**: En yüksek seviyedeki hata durumudur. Genellikle program daha fazla çalışamayacağı zaman kullanırız.

### Diğer basicConfig Parametreleri
**basicConfig** içerisine yazdığımız diğer parametrelerden de kısaca bahsetmek gerekirse:

**Filename**: Loglarımızın tutulacağı dosyanın ismi. Bir dizin belirtilmezse kendi bulunduğu dizinde tutar.

**Format**: Kaydın tutulacağı formatı belirler. (örneğimizde 3 parametre verilmiştir.)

**Filemode**: Logun tutulacağı modu belirler. “w” veya “a” tercih edebilirsiniz. “w” daha önceden yazılan dosyanın içeriğini yazmaya baştan başlarken, “a” dosyanın son kaldığı yerden devam eder.

Şimdi de benzer bir çıktıyı **print** fonksiyonu kullanarak almaya çalışmanın zorluğunu görmeyi deneyelim. 

In [3]:
# print.py isimli bir Python dosyası oluşturalım.
import datetime
try:
    result = 5 / 0
except Exception as e:
    print("{}: {}, {}, Exception thrown".format(type(e), e, datetime.datetime.now()))

<class 'ZeroDivisionError'>: division by zero, 2022-03-20 15:51:20.053352, Exception thrown


**format** fonksiyonunun içine yazdığımız değişkenlerini her birini, **print** fonksiyonunu kullandığımız her yerde tekrar tekrar yazmamız gerektiğini unutmayın. **logging** kütüphanesini kullandığımız senaryoda ise bunları kodumuzun başında bir kere tanımlamamız yeterli. **Print** fonksiyonu kullanarak oluşturduğumuz çıktıda hangi dizindeki kodun hangi satırının hata verdiği gibi detayların da yer almadığını belirtmek gerek.

## Logging ile Konsol Çıktısı Oluşturma

Bir diğer kullanım örneği olarak, çıktıyı bu kez bir dosyaya yazdırmak yerine direkt olarak ekranda görelim

In [4]:
# logging_example.py isimli bir Python dosyası oluşturalım.
import logging

logger = logging.getLogger()
console = logging.StreamHandler()

format_str = '%(asctime)s\t%(levelname)s -- %(processName)s %(filename)s:%(lineno)s -- %(message)s'
console.setFormatter(logging.Formatter(format_str))

logger.addHandler(console)

logger.setLevel(logging.ERROR) 
try:
    logger.debug("Process is started")
    result = 5 / 0
except Exception as e:
    logger.error("{} error occured, details in below".format(e), exc_info = True)
    print("You will see this line after full error details")

2022-03-20 15:51:20,090	ERROR -- MainProcess 3843085982.py:17 -- division by zero error occured, details in below
Traceback (most recent call last):
  File "/var/folders/9k/g_dyhr955x364hgp3rn4c7s40000gn/T/ipykernel_21268/3843085982.py", line 15, in <module>
    result = 5 / 0
ZeroDivisionError: division by zero


You will see this line after full error details


Kodumuz beklediğimiz gibi önce **logger**'ın çıktısını, sonra da **print** fonksiyonunun çıktısını ekrana yazdırdı.

Önce **getLogger** metoduyla **logging** kütüphanesinden **logger** değişkenimizi oluşturduk. Ardından çıktılarımızı konsola yazdırmak için **console** adıyla bir **handler** değişkeni oluşturduk. **StreamHandler** metodu, kayıtları **print**'in yaptığına benzer şekilde **stdout** aracılığıyla konsola gönderir.

**format_str** değişkeni ile birlikte oluşturmak istediğimiz hata mesajının formatını belirliyoruz. Bir önceki örneğimizden farklı olarak bu kez parametre sayısını artırdık ve 6 parametreyi görmek istediğimizi belirttik. Daha sonra **Formatter** metodu yardımıyla kaydımızı bu formata göre biçimlendirip bunu **setFormatter** metodunun yardımıyla konsola yazdırmak istediğimizi belirtiyoruz. Ardından **console** değişkenimizi de **logger**'a bir **handler** yardımıyla ekliyoruz. 

Görmek istediğimiz kayıt seviyesini **setLevel** metodu yardımıyla ayarlıyoruz. Bir önceki örnekte direkt olarak **basicConfig**'in içinde vermiştik. Alternatif olarak bu şekilde de ayarlayabiliriz.

Kodumuzu çalıştırdığımızda, görmek istediğimiz detayları eksiksiz bir şekilde konsol çıktısı olarak görüyoruz.

Son olarak, özellikle Machine Learning serisinde sıkça yaptığım gibi, **logging** kütüphanesinin dokümantasyon linkini paylaşmak istiyorum. Python geliştiricileri için, dokümantasyonun oldukça güzel yapıldığını ve okuyup uygulamanın çok geliştirdiğini belirtmekte fayda var: https://docs.python.org/3/howto/logging-cookbook.html#filters-contextual

Sonraki yazılarımızda görüşmek üzere.